# Graph 4: MultiConditional Graph


In [1]:
# imports
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [2]:
# StateSchema
class AgentState(TypedDict):
    num1: int
    num2: int
    operation: str
    result: int

In [3]:
# nodes (do not forget to add docstrings in nodes)
def addition_node(state: AgentState) -> AgentState:
    """
    Adds two numbers stored in the state.
    """
    state['result'] = state['num1'] + state['num2']
    return state

def subtraction_node(state: AgentState) -> AgentState:
    """
    Subtracts num2 from num1 stored in the state.
    """
    state['result'] = state['num1'] - state['num2']
    return state

def multiplication_node(state: AgentState) -> AgentState:
    """
    Multiplies two numbers stored in the state.
    """
    state['result'] = state['num1'] * state['num2']
    return state

In [4]:
# additional nodes for routing
def router(state: AgentState) -> AgentState:
    """
    Routes the state to the appropriate operation node based on the 'operation' field.
    """
    if state['operation'] == '+':
        return "addition_operation"
    elif state['operation'] == '-':
        return "subtraction_operation"
    elif state['operation'] == '*':
        return "multiplication_operation"
    else:
        return "end_node"

In [5]:
# StateGraph
graph = StateGraph(AgentState) 

graph.add_node("addition_node", addition_node)
graph.add_node("subtraction_node", subtraction_node)
graph.add_node("multiplication_node", multiplication_node)
graph.add_node("router", lambda state:state) # pass-through node for routing

graph.add_edge(START, "router")

graph.add_conditional_edges(
    "router",
    router,
    # "edge":"node"
    {
        "addition_operation": "addition_node",
        "subtraction_operation": "subtraction_node",
        "multiplication_operation": "multiplication_node",
        "end_node": END
    }
)

graph.add_edge("addition_node", END)
graph.add_edge("subtraction_node", END)
graph.add_edge("multiplication_node", END)

app = graph.compile()

In [6]:
# Visualize the graph
# from IPython.display import display, Image
# display(Image(app.get_graph().draw_mermaid_png()))

In [8]:
state_eg = AgentState(
    num1=10,
    num2=5,
    operation='*'
)
# Run the app with an example state
result = app.invoke(state_eg)
result

{'num1': 10, 'num2': 5, 'operation': '*', 'result': 50}